In [1]:
import datajoint as dj
import numpy as np
import datetime
import math

#from cloudvolume import CloudVolume
#from collections import Counter
#from funconnect import ta3

In [2]:
import cgal_Segmentation_Module as csm

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 10


# user: celiib
# pass: newceliipass
# host: at-database.ad.bcm.edu
# schemas: microns_% and celiib_%

In [4]:
schema = dj.schema('microns_ta3')
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')

Connecting celiib@10.28.0.34:3306


In [5]:
import os
import pathlib

##write the OFF file for the neuron
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd()
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing faces")
    #f.write("end")
    
    return str(path_and_filename),str(filename)

In [6]:
cm = ta3.CleansedMesh()
cm


segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio intended ratio of vertices and faces left after decimation,n_vertices,n_triangles,vertices,triangles,labeled_vertices Full array of labeled vertices.,"labeled_triangles Full array of labeled triangles/faces, useful for visualization."
1,331199,0.05,34714,69359,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,331199,0.35,245424,489615,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,421208,0.05,39896,79948,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,421208,0.35,283248,565934,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,481423,0.05,11708,23527,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,481423,0.35,82644,164871,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,579228,0.05,16667,33209,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,579228,0.35,121147,241383,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,694582,0.05,36820,74571,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,694582,0.35,262667,525352,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [7]:
"""my_key = dict(segmentation=1,segment_id=331199,decimation_ratio=0.35)

whole_neuron_dicts = dict()
whole_neuron_dicts[my_key['segment_id']] = (ta3.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=my_key['segment_id'])).fetch1()
                
verts = whole_neuron_dicts[my_key['segment_id']]["vertices"]

path_and_filename, off_file_name = write_Whole_Neuron_Off_file(whole_neuron_dicts[my_key["segment_id"]]["segment_id"],
                                            whole_neuron_dicts[my_key['segment_id']]["vertices"],
                                            whole_neuron_dicts[my_key['segment_id']]["triangles"])

print(path_and_filename)
print("my= " + str(off_file_name))
"""

'''
path_and_filename = "/Users/brendancelii/Google Drive/Xaq Lab/Final_Blender/whole_neuron_auto_label/neuron_331199"
off_file_name = "neuron_331199"

clusters = 2
smoothness = 0.4
csm.cgal_segmentation(path_and_filename,clusters,smoothness)

#print(verts)'''

'\npath_and_filename = "/Users/brendancelii/Google Drive/Xaq Lab/Final_Blender/whole_neuron_auto_label/neuron_331199"\noff_file_name = "neuron_331199"\n\nclusters = 2\nsmoothness = 0.4\ncsm.cgal_segmentation(path_and_filename,clusters,smoothness)\n\n#print(verts)'

In [8]:
cc = (ta3.CleansedMesh & "decimation_ratio=0.35" & [dict(segment_id=comp) for comp in [50467565,58045989,481423,579228,694582]])
cc



segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio intended ratio of vertices and faces left after decimation,n_vertices,n_triangles,vertices,triangles,labeled_vertices Full array of labeled vertices.,"labeled_triangles Full array of labeled triangles/faces, useful for visualization."
1,481423,0.35,82644,164871,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,579228,0.35,121147,241383,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,694582,0.35,262667,525352,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,50467565,0.35,299302,597998,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1,58045989,0.35,116266,232327,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [9]:
################# USE THIS FOR THE AUTOMATED PARAMETER TESTING OF THE SEGMENT
import cgal_Segmentation_Module as csm
import csv
import decimal
import time
import os

@schema
class ComponentAutoSegmentWhole(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3.CleansedMesh
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   
    
   """
    
    key_source = ta3.CleansedMesh & "decimation_ratio=0.35" #& [dict(segment_id=comp) for comp in [50467565,58045989]]#,481423,579228,694582]]
    
    whole_neuron_dicts = dict()
    
    def make(self, key):
        
        from cgal_Segmentation_Module import cgal_segmentation
        #key passed to function is just dictionary with the following attributes
        """segmentation
        segment_id
        decimation_ratio
        """
        
        
        #clusters_default = 18
        #smoothness_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
        #cluster_list = [2,3,4,5,6]
        
        smoothness_list = [0.2]
        cluster_list = [3]
          
        entire_neuron = (ta3.CleansedMesh & key).fetch1()
        neuron_ID = key["segment_id"]
        component_size = int(entire_neuron["n_triangles"])
        
        print("inside make function with " + str(neuron_ID))
        
        for smoothness in smoothness_list:
            
            for clusters in cluster_list:
                start_time = time.time()
                
                print(str(entire_neuron["segment_id"]) + " cluster:" + str(clusters) 
                      + " smoothness:" + str(smoothness))

                #generate the off file for each component
                #what need to send them:
                """----From cleansed Mesh---
                vertices
                triangles
                ----From component table--
                n_vertex_indices
                n_triangle_indices
                vertex_indices
                triangle_indices"""

                if key['segment_id'] not in self.whole_neuron_dicts:
                    self.whole_neuron_dicts[key['segment_id']] = (ta3.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
                
                
                path_and_filename, off_file_name = write_Whole_Neuron_Off_file(neuron_ID,
                                            self.whole_neuron_dicts[key['segment_id']]["vertices"],
                                            self.whole_neuron_dicts[key['segment_id']]["triangles"])
                
                print("About to start segmentation")
                
                #will have generated the component file by now so now need to run the segmentation
                cgal_segmentation(path_and_filename,clusters,smoothness)

                #generate the name of the files
                smoothness_str = str(smoothness)
                if(len(smoothness_str)<4):
                    smoothness_str = smoothness_str + "0"
                
                
                
                cgal_file_name = path_and_filename + "-cgal_" + str(clusters) + "_"+str(smoothness_str)
                group_csv_cgal_file = cgal_file_name + ".csv"
                sdf_csv_file_name = cgal_file_name+"_sdf.csv"


                #check if file actually exists
                import os
                exists = os.path.isfile(group_csv_cgal_file)
                
                if( not exists):
                    print("Segmentation not created for " + str(off_file_name))
                    print("################## " + str(neuron_ID) + " ##################")
                else:


                    with open(group_csv_cgal_file) as f:
                      reader = csv.reader(f)
                      your_list = list(reader)
                    group_list = []
                    for item in your_list:
                        group_list.append(int(item[0]))

                    with open(sdf_csv_file_name) as f:
                      reader = csv.reader(f)
                      your_list = list(reader)
                    sdf_list = []
                    for item in your_list:
                        sdf_list.append(float(item[0]))

                    #print(group_list)
                    #print(sdf_list)

                    #now write them to the datajoint table  
                    #table columns for ComponentAutoSegmentation: segmentation, segment_id, decimation_ratio, compartment_type, component_index, seg_group, sdf
                    comp_dict = dict(key,
                                        clusters=clusters,
                                        smoothness=smoothness,
                                        n_triangles=component_size,
                                        seg_group=group_list,
                                        sdf=sdf_list,
                                        median_sdf=np.median(sdf_list),
                                        mean_sdf=np.mean(sdf_list),
                                        third_q=np.percentile(sdf_list, 75),
                                        ninety_perc=np.percentile(sdf_list, 90),
                                        time_updated=str(datetime.datetime.now())[0:19])

                    self.insert1(comp_dict)

                    #then go and erase all of the files used: the sdf files, 
                    real_off_file_name = path_and_filename + ".off"

                    files_to_delete = [group_csv_cgal_file,sdf_csv_file_name,real_off_file_name]
                    for fl in files_to_delete:
                        if os.path.exists(fl):
                            os.remove(fl)
                        else:
                            print(fl + " file does not exist")

                    print("finished")
                    print("--- %s seconds ---" % (time.time() - start_time))
                

In [10]:
csm = ta3.ComponentAutoSegmentWhole()
csm
#csm.jobs
#schema.jobs.drop()
#csm.drop()
ComponentAutoSegmentWhole.populate(reserve_jobs=True)

inside make function with 421208
421208 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 442.260883808136 seconds ---
inside make function with 579228
579228 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 114.68654680252075 seconds ---
inside make function with 942272
942272 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 8.609437227249146 seconds ---
inside make function with 989772
989772 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 1.5346319675445557 seconds ---
inside make function with 1373380
1373380 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 82.55440998077393 seconds ---
inside make function with 1703823
1703823 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segment

inside make function with 52089750
52089750 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 314.5161259174347 seconds ---
inside make function with 53903126
53903126 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 127.83942604064941 seconds ---
inside make function with 54548179
54548179 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 385.8677718639374 seconds ---
inside make function with 55665174
55665174 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 18.99719214439392 seconds ---
inside make function with 56721274
56721274 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 248.84986090660095 seconds ---
inside make function with 58045989
58045989 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
A

finished
--- 20.63857412338257 seconds ---
inside make function with 119773356
119773356 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 6.52834677696228 seconds ---
inside make function with 120463949
120463949 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 36.751741886138916 seconds ---
inside make function with 121391399
121391399 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 273.5312247276306 seconds ---
inside make function with 122808597
122808597 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 219.7847409248352 seconds ---
inside make function with 128394576
128394576 cluster:3 smoothness:0.2
Done writing verts
Done writing faces
About to start segmentation
finished
--- 257.53680896759033 seconds ---


In [11]:
#len(ComponentAutoSegment())

In [12]:
#schema.jobs.delete()